In [ ]:
import numpy as np
import xarray as xr 
import seaborn as sns
import pandas as pd
import datetime
from netCDF4 import Dataset
import Nio
import Ngl
import matplotlib.pyplot as plt
import matplotlib
from mpl_toolkits.basemap import Basemap
import matplotlib.ticker as ticker
from matplotlib.patches import Polygon
from matplotlib.ticker import FormatStrFormatter

# Define helper functions

In [ ]:
# Define function to draw polygons on map
def draw_screen_poly(lats, lons, m):
    x, y = m(lons, lats)
    xy = zip(x,y)
    poly = Polygon(list(xy), fill=False, edgecolor='black',lw=3, alpha=0.5, linestyle='--')
    plt.gca().add_patch(poly)

In [ ]:
# Function to check if month is in November or December
def is_nd(month):
    return (month == 11) | (month == 12) 

In [ ]:
def read_cesm(filename, variable, subset=[288.0,320.0,-40.0,-10.0], plev=[850.],vinterp=False, SM=False, chop=True):
    """
    Function to read in CESM data and apply necessary processing steps.
    
    If a vertical variable (such as geopotential height) is being read, vertical interpolation is applied. Data are interpolated 
    to the levels defined by plev. 
    
    If soil moisture is being read, a weighted average is applied to get the root-zone average soil moisture.
    
    Parameters:
    -filename: String. Name of file to be read. Should be a NetCDF (.nc) file. 
    -variable: String. Refers to the desired CESM variable.
    -subset: List of floats. Format is [lon1, lon2, lat1, lat2]. Refers to the desired spatial domain to be extracted.
    -plev: Float. Vertical level(s) to which data should be interpolated - MUST BE IN MB.
    -vinterp: True or False. Should be set to True whenever vertical interpolation from model to pressure levels is necessary.
    -SM: True or False. Should be set to True if this function is being used to read in soil moisture data.
    -chop: True or False. If True, exclude the first time step from the file being read.
    
    Returns:
    An xarray DataArray with the data read from the file.  
    
    Author: Carolina Bieri (bieri2@illinois.edu)
    """
    print('Reading ' + filename)
    
    # Set reference pressure in mb
    P0 = 1000.
    
    # Open file containing desired variable; subset by lats and lons
    ds_var = xr.open_dataset(filename, chunks={}).sel(lat = slice(subset[2],subset[3]), lon = slice(subset[0],subset[1]))
  
    # Set time array as coordinate
    ds_var = ds_var.assign_coords(time = xr.CFTimeIndex(ds_var['time'].values).to_datetimeindex())
        
    # Remove the first day of output if necessary - some files already have the first day excluded
    # Must do this because the first day isn't really output
    if chop:
        if vinterp:
            # Read in surface pressure if vertical interpolation is necessary
            PS = ds_var['PS'].sel(time = slice('1979-01-02','2004-01-01'))
        
        ds_var = ds_var[variable].sel(time = slice('1979-01-02','2004-01-01'))
    else:
        ds_var = ds_var[variable]
    
    # CESM output files are written so that the corresponding timestep for daily output represents output
    # for the day before, so correct the time coordinate to reflect this
    ds_var['time'] = ds_var['time'] - pd.Timedelta(days = 1)

    # Perform vertical interpolation if necessary
    if vinterp:
        # Define coefficients necessary to perform vertical interpolation
        hyam = np.asarray([0.00364346569404006, 0.00759481964632869, 0.0143566322512925, 
                            0.0246122200042009, 0.0359232500195503, 0.0431937500834465, 
                            0.0516774989664555, 0.0615204982459545, 0.0737509578466415, 
                            0.0878212302923203, 0.103317126631737, 0.121547240763903, 
                            0.142994038760662, 0.168225079774857, 0.178230673074722, 
                            0.170324325561523, 0.161022908985615, 0.150080285966396, 
                            0.137206859886646, 0.122061938047409, 0.104244712740183, 
                            0.0849791541695595, 0.0665016956627369, 0.0501967892050743, 
                            0.037188658490777, 0.028431948274374, 0.0222089774906635, 
                            0.016407382208854, 0.0110745579004288, 0.00625495356507599, 
                            0.00198940909467638, 0])
        hybm = np.asarray([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0196774136275053, 
                            0.062504293397069, 0.112887907773256, 0.172161616384983, 
                            0.241894043982029, 0.323930636048317, 0.420442461967468, 
                            0.524799540638924, 0.624887734651566, 0.713207691907883, 
                            0.783669710159302, 0.831102818250656, 0.864811271429062, 
                            0.896237164735794, 0.92512384057045, 0.951230525970459, 
                            0.974335998296738, 0.992556095123291])
    
        # Perform vertical interpolation to desired pressure level(s)
        var_interp = Ngl.vinth2p(ds_var.values,hyam,hybm,np.asarray([plev]),PS.values,1,P0,1,False)
        
        if np.size(plev)==1:
            # Get rid of degenerate dimension if only interpolating to one level
            out = np.squeeze(var_interp[:,0,:,:])
        
            # Create Data Array to hold interpolated data
            out = xr.DataArray(out, coords={'time': ds_var['time'],'lat': ds_var['lat'], 
                                            'lon' : ds_var['lon']}, dims = ['time','lat','lon'])
        else:
            # Do this if interpolating to multiple levels
            out = xr.DataArray(var_interp, coords={'time': ds_var['time'],'lev': plev, 'lat': ds_var['lat'], 
                                                   'lon' : ds_var['lon']}, dims = ['time','lev','lat','lon'])
            
    # Process for extracting soil moisture data
    elif SM:
        # Read in SM data - first 8 layers (root zone)
        ds_var = ds_var[:,:8,:,:]
        
        # Create masked array before computing weighted average 
        masked_var = np.ma.masked_array(ds_var, np.isnan(ds_var))                
        
        # Define array of CLM soil layer thicknesses up to layer 8
        thick   = np.array([0.02,0.04,0.06,0.08,0.12,0.16,0.2,0.24])
        # Compute layer thickness weights 
        wgts    = thick/thick.sum()
        # Perform weighted average using weights
        wgt_avg = np.ma.average(masked_var,axis=1,weights=wgts)
        # Set weighted average SM data as output array
        out     = wgt_avg.filled(np.NaN)
        
        # Create DataArray
        out = xr.DataArray(out, coords={'time': ds_var['time'],'lat': ds_var['lat'], 
                                        'lon' : ds_var['lon']}, dims = ['time','lat','lon'])
        
    # Do this in all other cases
    else:
        out = ds_var
    

    return out

In [ ]:
# Helper function to subtract experiment from control and calculate monthly means
def means_and_diff(control, exp):
    
    # Resample data to monthly means from daily data
    control  = control.resample(time='M').mean(dim='time')
    exp      = exp.resample(time='M').mean(dim='time')
    
    # Get difference between experiment and control
    diff     = exp - control
    
    # Select differences for each month
    diff_oct = diff.sel(time=diff.time.dt.month==10)
    diff_nov = diff.sel(time=diff.time.dt.month==11)
    diff_dec = diff.sel(time=diff.time.dt.month==12)
    
    # Get means of differences for each month
    diff_comp_oct = diff_oct.mean(dim='time')
    diff_comp_nov = diff_nov.mean(dim='time')
    diff_comp_dec = diff_dec.mean(dim='time')
    
    return [diff_comp_oct, diff_comp_nov, diff_comp_dec]

In [ ]:
# Helper function to calculate fractional change between experiment and control
def calc_fracchng(change, control, subset=[298.0,313.0,-34.0,-19.0]):
    # Calculate fractional change
    frac_chng = change/control
    # Select within box
    frac_chng = frac_chng.sel(lat = slice(subset[2],subset[3]), lon = slice(subset[0],subset[1]))
    # Calculate mean
    frac_chng = frac_chng.mean()
    
    return frac_chng

In [ ]:
Open_Sans()

# Helper function to plot output
def plot_cesm(data, lat, lon, vmin, vmax, interval, ticks, cblabel, month, cmap, outfile, winds=False, U=None, V=None, plotnan=False):
    
    %matplotlib inline
    
    # Define lats and lons for box overlay
    lats_draw = [ -33.3, -18.3, -18.3, -33.3]
    lons_draw = [ 298.7, 298.7, 313.7, 313.7]
    
    fig = plt.figure(figsize=(10,10))

    map = Basemap(resolution='l', projection='cyl', llcrnrlon=min(lon), urcrnrlon=max(lon), 
                  llcrnrlat=min(lat), urcrnrlat=max(lat), lat_0=0, lon_0=0)

    # Define discrete colormap bins
    bins = np.arange(vmin,vmax+interval,interval)
        
    lon_plt, lat_plt = np.meshgrid(np.asarray(lon), np.asarray(lat))
    
    xi, yi = map(np.asarray(lon_plt), np.asarray(lat_plt))

    fill = map.contourf(xi, yi, data, vmin=vmin, vmax=vmax, levels=bins, cmap=cmap, extend='both')

    # Optionally plot winds
    if winds:
        plt.quiver(xi, yi, U, V, color='black',zorder=10, scale=30, headlength=3.0, headaxislength=3.0)
    
    # Optionally plot locations of masked NaN values
    if plotnan:
        wherenan = np.nonzero(np.isnan(data.values))
        xi_wherenan = xi[wherenan]
        yi_wherenan = yi[wherenan]
        plt.scatter(xi_wherenan+0.5,yi_wherenan+0.5,s=70,marker='x',color='black',alpha=0.5)

    map.drawcoastlines()
    map.drawcountries()

    cb = fig.colorbar(fill, orientation='horizontal', pad=0.05, ticks=ticks)
    cb.ax.tick_params(labelsize=26)
    cb.set_label(label=cblabel, size=26)

    # Draw box
    draw_screen_poly(lats_draw, lons_draw, map)
    
    plt.title(month, size=30)

    # Save plot
    plt.savefig(outfile + '.png', dpi=200)
    plt.show()

In [ ]:
%matplotlib inline
sns.set_style('ticks')

# Plot precipitation results

In [ ]:
# Define top level directory for CESM file locations
top_dir = '/glade/campaign/univ/uiuc0017/bieri/'

In [ ]:
# Read precip data from control simulation
pcp_control_c = read_cesm(top_dir + 'FHIST_SP_control/atm/hist/FHIST_SP_control.cam.h0.1979-2003.nc', 'PRECC')
pcp_control_l = read_cesm(top_dir + 'FHIST_SP_control/atm/hist/FHIST_SP_control.cam.h0.1979-2003.nc', 'PRECL')

In [ ]:
# Read precip data from dry experiment
pcp_dry_c     = read_cesm(top_dir + 'FHIST_SP_dry/atm/hist/FHIST_SP_dry.cam.h0.1979-2003.nc', 'PRECC')
pcp_dry_l     = read_cesm(top_dir + 'FHIST_SP_dry/atm/hist/FHIST_SP_dry.cam.h0.1979-2003.nc', 'PRECL')

In [ ]:
# Get total precip (convective + non-convective)
pcp_control_tot = pcp_control_c + pcp_control_l
pcp_dry_tot     = pcp_dry_c     + pcp_dry_l

In [ ]:
# Convert to mm/day
pcp_control_tot = pcp_control_tot*1000*86400
pcp_dry_tot     = pcp_dry_tot*1000*86400

In [ ]:
# Take monthly means and subtract control from experiment
pcp_diff_oct, pcp_diff_nov, pcp_diff_dec = means_and_diff(pcp_control_tot, pcp_dry_tot)

In [ ]:
# Select control data for November and take mean over all years
pcp_control_tot_nov = pcp_control_tot.sel(time=pcp_control_tot.time.dt.month==11)
pcp_control_tot_nov = pcp_control_tot_nov.mean(dim='time')

In [ ]:
# Select control data for December and take mean over all years
pcp_control_tot_dec = pcp_control_tot.sel(time=pcp_control_tot.time.dt.month==12)
pcp_control_tot_dec = pcp_control_tot_dec.mean(dim='time')

In [ ]:
# Calculate spatially averaged fractional change for Nov within box
frac_chng_pcp = calc_fracchng(pcp_diff_nov, pcp_control_tot_nov)

In [ ]:
frac_chng_pcp.values

In [ ]:
# Calculate spatially averaged fractional change for Dec within box
ac_chng_pcp_dec = calc_fracchng(pcp_diff_dec, pcp_control_tot_dec)

In [ ]:
frac_chng_pcp_dec.values

In [ ]:
# Plot output
plot_cesm(pcp_diff_nov, pcp_diff_nov.coords['lat'],  pcp_diff_nov.coords['lon'], vmin=-2.0, vmax=2.0, interval=0.31, ticks=[-2.0,-1.1,0,1.1,2.0],
          cblabel='Precipitation difference (mm/day)', month='NOV', cmap=matplotlib.colors.LinearSegmentedColormap.from_list("", ["#6c3811","white","#179DC9"]), outfile='diff_precip_nov')

In [ ]:
plot_cesm(pcp_diff_dec, pcp_diff_dec.coords['lat'], pcp_diff_dec.coords['lon'], vmin=-2.0, vmax=2.0, interval=0.31, ticks=[-2.0,-1.1,0,1.1,2.0],
          cblabel='Precipitation difference (mm/day)', month='DEC', cmap=matplotlib.colors.LinearSegmentedColormap.from_list("", ["#6c3811","white","#179DC9"]), outfile='diff_precip_dec')

# Plot temperature results

In [ ]:
# Read T2M from control and dry output
t_control = read_cesm(top_dir + 'FHIST_SP_control/atm/hist/FHIST_SP_control.cam.h0.1979-2003.nc', 'TREFHT')
t_dry     = read_cesm(top_dir + 'FHIST_SP_dry/atm/hist/FHIST_SP_dry.cam.h0.1979-2003.nc', 'TREFHT')

In [ ]:
# Get monthly means and subtract control from experiment
t_diff_oct, t_diff_nov, t_diff_dec = means_and_diff(t_control, t_dry)

In [ ]:
# Get control data for Nov and take average over all years
t_control_nov = t_control.sel(time=t_control.time.dt.month==11)
t_control_nov = t_control_nov.mean(dim='time')

In [ ]:
# Get control data for Dec and take average over all years
t_control_dec = t_control.sel(time=t_control.time.dt.month==12)
t_control_dec = t_control_dec.mean(dim='time')

In [ ]:
# Convert to Celsius
t_control_nov = t_control_nov-273.15
t_control_dec = t_control_dec-273.15

In [ ]:
# Compute spatially averaged fractional change for Nov
frac_chng_t   = calc_fracchng(t_diff_nov, t_control_nov)

In [ ]:
frac_chng_t.values

In [ ]:
# Compute spatially averaged fractional change for Dec
frac_chng_t_dec   = calc_fracchng(t_diff_dec, t_control_dec)

In [ ]:
frac_chng_t_dec.values

In [ ]:
plot_cesm(t_diff_nov, t_diff_nov.coords['lat'], t_diff_nov.coords['lon'], vmin=-3.7, vmax=3.7, interval=0.57, ticks=[-3.7,-2,0,2,3.7],
          cblabel='Temperature difference (K)', month='NOV', cmap=matplotlib.colors.LinearSegmentedColormap.from_list("", ["#131DBF","white","#C91717"]), outfile='diff_t_nov')

In [ ]:
plot_cesm(t_diff_dec, t_diff_dec.coords['lat'], t_diff_dec.coords['lon'], vmin=-3.7, vmax=3.7, interval=0.57, ticks=[-3.7,-2,0,2,3.7],
          cblabel='Temperature difference (K)', month='DEC', cmap=matplotlib.colors.LinearSegmentedColormap.from_list("", ["#131DBF","white","#C91717"]), outfile='diff_t_dec')

# Plot LH results

In [ ]:
# Read LH control and dry output
lh_control = read_cesm(top_dir + 'FHIST_SP_control/lnd/hist/FHIST_SP_control.clm2.h0.1979-2003.nc', 'EFLX_LH_TOT')
lh_dry     = read_cesm(top_dir + 'FHIST_SP_dry/lnd/hist/FHIST_SP_dry.clm2.h0.1979-2003.nc', 'EFLX_LH_TOT', chop = False)

In [ ]:
# Calculate difference and take mean over all years 
lh_diff_oct, lh_diff_nov, lh_diff_dec = means_and_diff(lh_control, lh_dry)

In [ ]:
# Get control data for Nov and take mean over all years
lh_control_nov = lh_control.sel(time=lh_control.time.dt.month==11)
lh_control_nov = lh_control_nov.mean(dim='time')

In [ ]:
# Get control data for Dec and take mean over all years
lh_control_dec = lh_control.sel(time=lh_control.time.dt.month==12)
lh_control_dec = lh_control_dec.mean(dim='time')

In [ ]:
# Calculate spatially averaged LH change for Nov
frac_chng_lh = calc_fracchng(lh_diff_nov, lh_control_nov)

In [ ]:
# Calculate spatially averaged LH change for Dec
frac_chng_lh_dec = calc_fracchng(lh_diff_dec, lh_control_dec)

In [ ]:
frac_chng_lh_dec.values

In [ ]:
plot_cesm(lh_diff_nov, lh_diff_nov.coords['lat'], lh_diff_nov.coords['lon'], vmin=-95, vmax=95, interval=14.7, ticks=[-95,-51,0,51,95],
          cblabel='LH difference ($W/m^2$)', month='NOV', cmap=matplotlib.colors.LinearSegmentedColormap.from_list("", ["#131DBF","white","#C91717"]), outfile='diff_lh_nov')

In [ ]:
plot_cesm(lh_diff_dec, lh_diff_dec.coords['lat'], lh_diff_dec.coords['lon'], vmin=-95, vmax=95, interval=14.7, ticks=[-95,-51,0,51,95],
          cblabel='LH difference ($W/m^2$)', month='DEC', cmap=matplotlib.colors.LinearSegmentedColormap.from_list("", ["#131DBF","white","#C91717"]), outfile='diff_lh_dec')

# Plot SH results

In [ ]:
sh_control = read_cesm(top_dir + 'FHIST_SP_control/lnd/hist/FHIST_SP_control.clm2.h0.1979-2003.nc', 'FSH')
sh_dry     = read_cesm(top_dir + 'FHIST_SP_dry/lnd/hist/FHIST_SP_dry.clm2.h0.1979-2003.nc', 'FSH', chop = False)

In [ ]:
sh_diff_oct, sh_diff_nov, sh_diff_dec = means_and_diff(sh_control, sh_dry)

In [ ]:
sh_control_nov = sh_control.sel(time=sh_control.time.dt.month==11)
sh_control_nov = sh_control_nov.mean(dim='time')

In [ ]:
sh_control_dec = sh_control.sel(time=sh_control.time.dt.month==12)
sh_control_dec = sh_control_dec.mean(dim='time')

In [ ]:
frac_chng_sh = calc_fracchng(sh_diff_nov, sh_control_nov)

In [ ]:
frac_chng_sh_dec = calc_fracchng(sh_diff_dec, sh_control_dec)

In [ ]:
frac_chng_sh_dec.values

In [ ]:
plot_cesm(sh_diff_nov, sh_diff_nov.coords['lat'], sh_diff_nov.coords['lon'], vmin=-95, vmax=95, interval=14.7, ticks=[-95,-51,0,51,95],
          cblabel='SH difference ($W/m^2$)', month='NOV', cmap=matplotlib.colors.LinearSegmentedColormap.from_list("", ["#131DBF","white","#C91717"]), outfile='diff_sh_nov')

In [ ]:
plot_cesm(sh_diff_dec, sh_diff_dec.coords['lat'], sh_diff_dec.coords['lon'], vmin=-95, vmax=95, interval=14.7, ticks=[-95,-51,0,51,95],
          cblabel='SH difference ($W/m^2$)', month='DEC', cmap=matplotlib.colors.LinearSegmentedColormap.from_list("", ["#131DBF","white","#C91717"]), outfile='diff_sh_dec')

# Plot GH and U, V results

In [ ]:
u_control = read_cesm(top_dir + 'FHIST_SP_control/atm/hist/FHIST_SP_control.cam.h0.1979-2003.nc', 'U', plev=850., vinterp = True)
u_dry     = read_cesm(top_dir + 'FHIST_SP_dry/atm/hist/FHIST_SP_dry.cam.h0.1979-2003.nc', 'U',  plev=850., vinterp = True)

In [ ]:
v_control = read_cesm(top_dir + 'FHIST_SP_control/atm/hist/FHIST_SP_control.cam.h0.1979-2003.nc', 'V', plev=850., vinterp = True)
v_dry     = read_cesm(top_dir + 'FHIST_SP_dry/atm/hist/FHIST_SP_dry.cam.h0.1979-2003.nc', 'V', plev=850., vinterp = True)

In [ ]:
# Exclude bad values
u_control = u_control.where(u_control<100)
u_dry     = u_dry.where(u_dry<100)
v_control = v_control.where(v_control<100)
v_dry     = v_dry.where(v_dry<100)

In [ ]:
u_diff_oct, u_diff_nov, u_diff_dec = means_and_diff(u_control, u_dry)
v_diff_oct, v_diff_nov, v_diff_dec = means_and_diff(v_control, v_dry)

In [ ]:
gh_control = read_cesm(top_dir + 'FHIST_SP_control/atm/hist/FHIST_SP_control.cam.h0.1979-2003.nc', 'Z3', plev=850., vinterp = True)
gh_dry     = read_cesm(top_dir + 'FHIST_SP_dry/atm/hist/FHIST_SP_dry.cam.h0.1979-2003.nc', 'Z3', plev=850., vinterp = True)

In [ ]:
# Exclude bad values
gh_control = gh_control.where(gh_control < 10000)
gh_dry     = gh_dry.where(gh_dry < 10000)

In [ ]:
gh_diff_oct, gh_diff_nov, gh_diff_dec = means_and_diff(gh_control, gh_dry)

In [ ]:
gh_diff_dec = gh_diff_dec.where(gh_diff_dec < 4)

In [ ]:
plot_cesm(gh_diff_nov, gh_diff_nov.coords['lat'], gh_diff_nov.coords['lon'], vmin=-6.0, vmax=6.0, interval = 0.925, ticks = [-6, -3.2, 0, 3.2, 6],
          cblabel='GH difference (m)', month='NOV', cmap=matplotlib.colors.LinearSegmentedColormap.from_list("", ["#131DBF","white","#C91717"]), outfile='diff_gh_nov',
          winds=True, U=u_diff_nov, V=v_diff_nov, plotnan=True)

In [ ]:
plot_cesm(gh_diff_dec, gh_diff_dec.coords['lat'], gh_diff_dec.coords['lon'], vmin=-6.0, vmax=6.0, interval = 0.925, ticks = [-6, -3.2, 0, 3.2, 6],
          cblabel='GH difference (m)', month='DEC', cmap=matplotlib.colors.LinearSegmentedColormap.from_list("", ["#131DBF","white","#C91717"]), outfile='diff_gh_dec',
          winds=True, U=u_diff_dec, V=v_diff_dec, plotnan=True)

# Plot SM results

In [ ]:
sm_control = read_cesm(top_dir + 'FHIST_SP_control/lnd/hist/FHIST_SP_control.clm2.h0.1979-2003.nc', 'H2OSOI', SM=True)
sm_dry     = read_cesm(top_dir + 'FHIST_SP_dry/lnd/hist/FHIST_SP_dry.clm2.h0.1979-2003.nc', 'H2OSOI', chop = False, SM=True)

In [ ]:
sm_diff_oct, sm_diff_nov, sm_diff_dec = means_and_diff(sm_control, sm_dry)

In [ ]:
plot_cesm(sm_diff_nov,sm_diff_nov.coords['lat'], sm_diff_nov.coords['lon'], vmin=-0.2, vmax=0.2, interval=0.031, ticks=[-0.2,-0.11,0,0.11,0.2],
          cblabel='SM difference ($m^3/m^3$)', month='NOV', cmap=matplotlib.colors.LinearSegmentedColormap.from_list("", ["#6c3811","white","#21A926"]), outfile='diff_sm_nov')

In [ ]:
plot_cesm(sm_diff_dec, sm_diff_dec.coords['lat'], sm_diff_dec.coords['lon'], vmin=-0.2, vmax=0.2, interval=0.031, ticks=[-0.2,-0.11,0,0.11,0.2],
          cblabel='SM difference ($m^3/m^3$)', month='DEC', cmap=matplotlib.colors.LinearSegmentedColormap.from_list("", ["#6c3811","white","#21A926"]), outfile='diff_sm_dec')

# Additional GH analysis

In [ ]:
# Define subset for spatial average
subset = [298.0,313.0,-34.0,-19.0]

In [ ]:
# Average over lat-lon subsets and take difference
gh_control_ext = gh_control.sel(lat = slice(subset[2],subset[3]), lon = slice(subset[0],subset[1]))
gh_dry_ext     = gh_dry.sel(lat = slice(subset[2],subset[3]), lon = slice(subset[0],subset[1]))
gh_control_ts  = gh_control_ext.mean(dim='lat', skipna=True).mean(dim='lon', skipna=True)
gh_dry_ts      = gh_dry_ext.mean(dim='lat').mean(dim='lon')
gh_diff_ts     = gh_dry_ts - gh_control_ts

In [ ]:
# Average over lat-lon subsets and take difference
t_control_ext  = t_control.sel(lat = slice(subset[2],subset[3]), lon = slice(subset[0],subset[1]))
t_dry_ext      = t_dry.sel(lat = slice(subset[2],subset[3]), lon = slice(subset[0],subset[1]))
t_control_ts   = t_control_ext.mean(dim='lat', skipna=True).mean(dim='lon', skipna=True)
t_dry_ts       = t_dry_ext.mean(dim='lat').mean(dim='lon')
t_diff_ts      = t_dry_ts - t_control_ts

In [ ]:
# Get average values for each day averaged over all years
gh_diff_ts.coords['monthday'] = (
    ('time', ),
    pd.MultiIndex.from_arrays([gh_diff_ts.time.dt.month, gh_diff_ts.time.dt.day]))

days_gh = gh_diff_ts.groupby('monthday').mean(dim='time')

In [ ]:
t_diff_ts.coords['monthday'] = (
    ('time', ),
    pd.MultiIndex.from_arrays([t_diff_ts.time.dt.month, t_diff_ts.time.dt.day]))

days_t = t_diff_ts.groupby('monthday').mean(dim='time')

In [ ]:
# Plot time series
days = np.arange(0,61)

fig, ax1 = plt.subplots(figsize = (10,5))

ax1.set_xlabel('Days after Nov 1', fontsize = 16)
ax1.set_ylabel('850 hPa height difference (m)', fontsize = 16, color = 'darkviolet')
#ax1.plot(days,trendpoly_gh(days), '--', color='black', lw=2.5)
ax1.plot(days, days_gh[is_nd(days_gh.monthday_level_0).values], color = 'darkviolet', lw = 3)
ax1.hlines(0, 0, 60, alpha = 0.7)
ax1.vlines(30, -20, 20, alpha = 0.7, linestyles = 'dashed')
ax1.set_xlim(0, 60)
ax1.set_ylim(-20, 20)
ax1.tick_params(axis = 'y', labelcolor = 'darkviolet', labelsize = 14)
ax1.tick_params(axis = 'x', labelsize = 14)

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

ax2.set_ylabel('2-m temp. difference (K)', fontsize = 16, color = 'lightseagreen')  # we already handled the x-label with ax1
#ax2.plot(days,trendpoly_t(days), '--', color='lightgreen', lw=2.5)
ax2.plot(days,days_t[is_nd(days_t.monthday_level_0).values], color = 'lightseagreen', lw = 3)
ax2.set_ylim(-5, 5)
ax2.tick_params(axis = 'y', labelcolor = 'lightseagreen', labelsize = 14)

plt.title('Time series of 850 hPa geo. hgt. and 2-m temp. differences\n Nov 1  to Dec 31', fontsize=20)
plt.savefig('gh_t_ts.pdf', dpi=150)
plt.show()

In [ ]:
# Get GH vertical profiles for control and dry
gh_vert_con = read_cesm(top_dir + 'FHIST_SP_control/atm/hist/FHIST_SP_control.cam.h0.1979-2003.nc', 'Z3', subset, plev=np.arange(225.,925.,25), vinterp=True)
gh_vert_dry = read_cesm(top_dir + 'FHIST_SP_dry/atm/hist/FHIST_SP_dry.cam.h0.1979-2003.nc', 'Z3', subset, plev=np.arange(225.,925.,25), vinterp=True)

In [ ]:
# Take difference and mean over all years
gh_vert_con = gh_vert_con.mean(dim='lat').mean(dim='lon')
gh_vert_dry = gh_vert_dry.mean(dim='lat').mean(dim='lon')
gh_vert_diff = gh_vert_dry-gh_vert_con

In [ ]:
# Exclude bad values
gh_vert_diff = gh_vert_diff.where(gh_vert_diff < 10000)
gh_vert_diff = gh_vert_diff.where(gh_vert_diff > -10000)

In [ ]:
# Get relative difference
gh_vert_diff = gh_vert_diff/gh_vert_con

In [ ]:
# Get means for each day of the year
gh_vert_diff.coords['monthday'] = (
    ('time', ),
    pd.MultiIndex.from_arrays([gh_vert_diff.time.dt.month, gh_vert_diff.time.dt.day]))

days_gh_vert = gh_vert_diff.groupby('monthday').mean(dim='time')

In [ ]:
Plot vertical profile with time
%matplotlib inline
sns.set_style('ticks')
f2 = plt.figure(figsize=(10,5))
cs = plt.contourf(np.arange(0,60), days_gh_vert['lev'], days_gh_vert[306:,:].transpose(),  levels=np.linspace(-0.008,0.008,10), cmap="bwr",
                 extend='both', alpha=0.7)
plt.gca().invert_yaxis()
cb = plt.colorbar(cs, boundaries=(-0.1,0.1), label='Fractional change', format='%4.3f')
plt.xlabel('Days after Nov 1', fontsize = 16)
plt.ylabel('Pressure (hPa)', fontsize = 16)
plt.vlines(30, 900, 225, alpha = 0.7, linestyles = 'dashed')
plt.title('Fractional change in geopotential height \n Nov 1 to Dec 31', fontsize = 20)
plt.tick_params(axis = 'y', labelsize = 14)
plt.tick_params(axis = 'x', labelsize = 14)
cb.ax.tick_params(labelsize=12)
cb.set_label(label = 'Fractional change', size = 14)
plt.savefig('gh_chng.pdf', dpi=150)
plt.show()